# Running a job

## Openfoam

We will run an example job using the OpenFoam 7 module for 

`OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7`

Let's load the module:
```bash
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7
```

Source the OpenFoam bashrc:

```bash
source $foamDotFile```

Lets switch to our scratch space:

```bash
cd $SCRATCH
```


And copy over the run files:
``` bash
cp -r $WM_PROJECT_DIR/tutorials/multiphase/interFoam/laminar/damBreak ./```

An example run script:
    
```bash
#!/bin/bash
#SBATCH --account=prj3_phase1
#SBATCH --job-name=AMD
#SBATCH --output=AMD.%j.o
#SBATCH --error=AMD.%j.e
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --partition=spot-hbv3-16
#SBATCH --time=04:00:00

module purge
module load OpenFoam_hbv3/gcc_9.2.0/openmpi_4.1.0/v7

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)

echo $campaigndir | if grep -q campagin; then 
 workdir=$( pwd | sed 's/campaign/scratch/');
 # Copy any inputs required to the work directory:
 cp $campaigndir/* $workdir/;
 cd $workdir;
 echo  "Work directory" $workdir ;
else
 echo "Not submitting from campaign. Check your submission script."
 exit 1;
fi

. $foamDotFile
./Allrun

# Copy back results.
srun cp -Rf $workdir/*  $campaigndir/
#Clean up scratch
rm -rf $workdir
```

Lets switch into the folder 
```bash
cd damBreak
```

And after copying the above runscript into `run_job.slm` lets submit the job:

```bash
sbatch run_job.slm
```

## ANSYS

```bash
#!/bin/bash
#SBATCH --job-name=Free_Surf
#SBATCH --account=sysadmin
#SBATCH --partition=spot-hc-44
#SBATCH --time=2:00:00
#SBATCH --nodes=1
#SBATCH --tasks-per-node=44

env | grep SLURM | sort -d
echo

export CASE="Free_Surf"

## Load the required modules
module load ansys/v212
module list

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)

echo $campaigndir | if grep -q campagin; then 
 workdir=$( pwd | sed 's/campaign/scratch/');
 # Copy any inputs required to the work directory:
 cp $campaigndir/* $workdir/
 cd $workdir
 echo  "Work directory" $workdir ;
else
 echo "Not submitting from campaign. Check your submission script."
 exit 1;
fi


## Define files
export DEFFILE="${CASE}.def"

## Define additional options
OPTIONS="-double -batch"

CFX_PARALLEL="$CFX_BATCH_MODE $CFX_CCL_FILE $CFX_START_METHOD $CFX_PARTITION $CFX_PARTITIONER"

HOSTS=$(srun --ntasks-per-node $SLURM_CPUS_ON_NODE hostname | sort | uniq -c | awk '{print $2 "*" $1}' | paste -sd, -)
echo "HOSTS: $HOSTS"
echo ""

# Run CFX5SOLVE
cfx5solve -def Free_Surf.def ${OPTIONS} -start-method 'Intel MPI Distributed Parallel' -par-dist $HOSTS -partitioner ${CFXPARTITIONER}

# Copy back results.
srun cp -Rf $workdir/*  $campaigndir/
#Clean up scratch
rm -rf $workdir
```

### Matlab

A hello world matlab script 'hello_world.m':

```matlab
disp('Hello, World');
```

An example run-script for a matlab job, which 
when submitted from /campaign:

```bash
#!/bin/bash
# Account to charge to
#SBATCH --account=prj4_phase1
#Name of the job
#SBATCH --job-name=Matlab
#SBATCH --output=Matlab.%j.o
#SBATCH --error=Matlab.%j.e
# Number of nodes requested
#SBATCH --nodes=1
# The no of tasks per node
#SBATCH --ntasks-per-node=44
# The partition to submit to
#SBATCH --partition=spot-hc-44
# The runtime
#SBATCH --time=04:00:00


# Purge modules and load matlab
module purge
module load matlab/2020b

# As an example of moving input data from /campaign
# to scratch we will create and mirror the submit
# directory structure on scratch:

campaigndir=$(pwd)

echo $campaigndir | if grep -q campagin; then 
 workdir=$( pwd | sed 's/campaign/scratch/');
 # Copy any inputs required to the work directory:
 cp $campaigndir/* $workdir/;
 cd $workdir;
 echo  "Work directory" $workdir ;
else
 echo "Not submitting from campaign. Check your submission script."
 exit 1;
fi



matlab -nosplash -nodisplay -r "hello_world; exit"

# Copy back results.
srun cp -Rf $workdir/*  $campaigndir/
#Clean up scratch
rm -rf $workdir
```